# Ensemble learning

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score

## Zbiór danych

W zadaniu poddany analizie będzie dobrze już znany zbiór MNIST.

In [2]:
mnist = fetch_openml("mnist_784", data_home="./mnist_784", cache=True)

In [3]:
X = mnist.data[:1000]
y = mnist.target[:1000]

## Transformacja zbioru do przestrzeni 30D

In [4]:
standard_scaler = StandardScaler()
X_scaled = standard_scaler.fit_transform(X)

pca = PCA(30)
X_pca = pca.fit_transform(X_scaled)

## Wybór trudno rozdzielalnych cech

W celu znalezienia trudno rozróżnialnych cech zbioru możemy skorzystać z metody t-SNE.

In [ ]:
X_embedded = TSNE(n_components=2).fit_transform(X_pca)
X_embedded.shape

In [ ]:
ids = [str(i) for i in range(10)]

for c, label in zip(mcolors.TABLEAU_COLORS, ids):
    plt.scatter(X_embedded[y == label, 0], X_embedded[y == label, 1], c=c, label=label)
plt.legend()
plt.show()

Wyraźnie widać zgrupowanie klasy 4 i 9. Użyjemy ich w dalszej części zadania.

In [ ]:
digit_filter = np.where((y == '4') | (y == '9'))
X_filtered, y_filtered = X_pca[digit_filter], y[digit_filter]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=13)

## Bagging

W zależności od sposobu wyboru podzbiorów zbioru danych wyróżniamy różne rodzaje tego algorytmu:

- **pasting** - wybór losowych podzbiorów próbek
- **bagging** - wybór losowych podzbiorów próbek ze zwracaniem
- **random subspaces** - wybór losowych podzbiorów cech
- **random patches** - wybór losowych podzbiorów próbek i cech

### Klasyczny bagging

In [ ]:
clf = BaggingClassifier(base_estimator=SVC(), n_estimators=5, max_samples=0.5).fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, clf.predict(X_test))

### Random Forest

In [ ]:
clf = RandomForestClassifier(n_estimators=5, max_samples=0.5).fit(X_train, y_train)
clf.base_estimator = base_estimator=SVC()

In [ ]:
accuracy_score(y_test, clf.predict(X_test))

### Ada Boost

In [ ]:
clf = AdaBoostClassifier(n_estimators=5).fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, clf.predict(X_test))

### Random subspace

In [ ]:
clf = BaggingClassifier(base_estimator=SVC(),
                        n_estimators=5,
                        bootstrap_features=True,
                        max_samples=0.5).fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, clf.predict(X_test))